# DATA_PATH

In [ ]:
import os

# Work in PC, Kaggle, Colab
IN_PC = ('VSCODE_CWD' in os.environ) and (not 'AK_KAGGLE_ENV' in os.environ)
IN_DOCKER = ('AK_KAGGLE_ENV' in os.environ)
IN_COLAB = ('COLAB_GPU' in os.environ)
# DATA_PATH = "../../data" if os.path.exists("../../data") and IN_PC else "../../../data" if os.path.exists("../../../data") and IN_PC else "/kaggle"

if IN_PC:
    DATA_PATH = os.path.abspath("../../data") if os.path.isdir("../../data") else os.path.abspath("../../../data")
elif IN_COLAB:
    DATA_PATH = "/content/drive/MyDrive/Colab_Kaggle/data"
else:
    DATA_PATH = "/kaggle"
    

# for dirname, _, filenames in os.walk(DATA_PATH + '/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
# shutil.copy(DATA_PATH + '/input/00000-01/fix.db', DATA_PATH + '/working/fix.db')

# Colab

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive');

# !pip install colab_ssh --upgrade
# from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
# launch_ssh_cloudflared(password="1")

In [ ]:
# !pip install -q datatable
# !pip install -q catboost
# !pip install -q optuna
# !pip install -q kaggle

if IN_COLAB:
    !pip install -q torchsummary
    !pip install -q torchsummaryX
    !pip install -q pytorch-model-summary
    !pip install -q torchinfo

# !git clone --recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# # # # !sudo apt-get -y install python-pip
# # # # !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install — precompile

# Imports

In [ ]:
import os
import numpy as np 
import pandas as pd
import time
import gc
import warnings

import joblib
import plotly.express as px
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from copy import deepcopy

from sklearn.model_selection import *
from sklearn.impute import *
from sklearn.preprocessing import *
from sklearn.metrics import *

from tqdm.notebook import tqdm

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('float_format', '{:f}'.format)
pd.set_option('float_format', '{:,}'.format)
warnings.filterwarnings('ignore')

# Variables

In [ ]:
n_splits = 5
target_col='Cover_Type' # name of target column
id_col='Id'
competition='tps1221'
fullname_competition ='tabular-playground-series-dec-2021'

WORKING_PATH = DATA_PATH + '/working/'+ competition + '/'
PKL_PATH = DATA_PATH + '/working/'+ competition + '/pkl/'
BOOSTERS_PATH = PKL_PATH + 'boosters/'

# Load Data
* 🙏 [[TPS-1221] Load data 50.36% faster](https://www.kaggle.com/akmeghdad/tps-1221-load-data-50-36-faster)

In [ ]:
%%time

if IN_PC or IN_COLAB:
    train = joblib.load(PKL_PATH + 'train-32.pkl')
    test = joblib.load(PKL_PATH + 'test-32.pkl').drop(columns=[id_col])
    submission = joblib.load(PKL_PATH + 'submission-32.pkl')
else:
    train = joblib.load(DATA_PATH + '/input/tps1221data/' + 'train-32.pkl')
    test = joblib.load(DATA_PATH + '/input/tps1221data/' + 'test-32.pkl').drop(columns=[id_col])
    submission = joblib.load(DATA_PATH + '/input/tps1221data/' + 'submission-32.pkl')

# EDA

# Exploring Train Data

In [ ]:
target_df = pd.DataFrame(train[target_col].value_counts()).reset_index()
target_df.columns = [target_col, 'count']
fig = px.bar(data_frame =target_df, 
             x = 'Cover_Type',
             y = 'count' , 
             color = "count",
             color_continuous_scale="Emrld") 
fig.show()
# target_df.sort_values(by =target_col , ignore_index = True)

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(x=train[target_col])
plt.title("Exploring Train Data")
plt.xlabel("Cover Type")
ax.bar_label(ax.containers[0])
plt.show()

# Features Correlation

In [ ]:
fig, ax = plt.subplots(figsize=(12 , 12))
corr = train.drop(columns=[id_col]).corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr,
        square=True, center=0, linewidth=0.2,
        cmap=sns.diverging_palette(250, 20, as_cmap=True),
        mask=mask, ax=ax) 
ax.set_title('Feature Correlation', loc='left', fontweight='bold')
plt.show()

In [ ]:
train.iloc[:, :-1].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='GnBu')\
                     .bar(subset=["max"], color='#BB0000')\
                     .bar(subset=["mean",], color='green')